# ATPESC 2019 REINFORCEMENT LEARNING TUTORIAL

Presented by Sami Khairy

#### Tutorial Outline
In this tutorial we will solve the CartPole-v1 classic control problem, using three RL algorithms:
1. Tabular Q-Learning
2. Deep Q-Networks (DQN)
3. Simple Policy Gradient

#### Problem Description
"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center."

Source: https://gym.openai.com/envs/CartPole-v1/


In [1]:
#### Import necessary packages
import tensorflow as tf
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# Environment Creation

In [2]:
env = gym.make("CartPole-v1")

In [3]:
help(env.env)

Help on CartPoleEnv in module gym.envs.classic_control.cartpole object:

class CartPoleEnv(gym.core.Env)
 |  Description:
 |      A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum starts upright, and the goal is to prevent it from falling over by increasing and reducing the cart's velocity.
 |  
 |  Source:
 |      This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson
 |  
 |  Observation: 
 |      Type: Box(4)
 |      Num     Observation                 Min         Max
 |      0       Cart Position             -4.8            4.8
 |      1       Cart Velocity             -Inf            Inf
 |      2       Pole Angle                 -24 deg        24 deg
 |      3       Pole Velocity At Tip      -Inf            Inf
 |      
 |  Actions:
 |      Type: Discrete(2)
 |      Num     Action
 |      0       Push cart to the left
 |      1       Push cart to the right
 |      
 |  

# Random Action Selection Strategy

In [4]:
observation = env.reset()
episode_cnt = 0
episode_ret = 0
time_steps = 0
episode_returns_random = []
num_episodes = 200
while episode_cnt < num_episodes:
    # env.render()
    # sample random actions from your action space
    action = env.action_space.sample() 
    observation, reward, done, info = env.step(action)
    time_steps += 1
    episode_ret += reward
    if done:
        #Reached a terminal state
        observation = env.reset()
        episode_cnt += 1
        episode_returns_random.append(episode_ret)
        episode_ret = 0

In [5]:
print("Performance of Random Policy.")
print("Number of Test Episodes {}.".format(episode_cnt))
print("Expected Episode Length: {}.".format(time_steps/episode_cnt))
print("Expected Episode Return: {}.".format(np.mean(episode_returns_random)))

Performance of Random Policy.
Number of Test Episodes 200.
Expected Episode Length: 23.165.
Expected Episode Return: 23.165.


# Tabular Q-Learning

Remark: The state space of CartPole-v1 is a vector in $R^4$. In order to use a tabular Q-learning method, we need to discretize the state space. 

In order to have a finite number of discrete states, I map the R^4 range into [-1,1]^4 using tanh.

In [6]:
#this function takes the state in R^4, maps it to [-1,1]^4 by applying tanh, discretize the space,
#and returns index of the state in multidimensional space
def state_to_multi_index(state,step_size=0.2):
    state = np.tanh(np.asarray(state))
    assert np.sum(-1 <= state)+ np.sum(1 >= state) == 2*len(state)
    positive_state = state + 1
    state_index = []
    for s_dim in positive_state:
        state_index.append(int(np.ceil(s_dim/step_size)))
    return state_index

In [7]:
#example,
s_temp = np.asarray([-10,3, 1.1,2.1, -2])
print(state_to_multi_index(state=s_temp,step_size=0.1))

[1, 20, 19, 20, 1]


In [8]:
#this function converts the index in a multidimensional array to a linear index, so we can have a 2D Q-table later on
def convert(s, step_size=0.2):
    s_id = state_to_multi_index(s,step_size)
    max_l = 2/step_size + 1
    assert np.sum(0 <= np.asarray(s_id))+ np.sum(max_l >= np.asarray(s_id)) == 2*len(s)
    linear_index = 0
    for i in range(len(s)):
        linear_index += s_id[i]*(max_l**i)
    return int(linear_index)

In [9]:
step_size = 0.1

In [10]:
max_l = 2/step_size +1
number_of_states = int(max_l**env.observation_space.shape[0])

In [11]:
number_of_states

194481

In [12]:
#Initialize Q-table with all zeros
Q = np.zeros([number_of_states,env.action_space.n])
#Set learning parameters
lr = .8
gamma = .95
num_episodes = 2000
#list to contain total rewards and steps per episode
ep_len = []
ep_ret = []
epsilon = 1
for i in tqdm(range(num_episodes)):
    #Reset environment and get first state
    s_ = env.reset()
    s = convert(s_,step_size)
    curr_ep_ret = 0
    terminal = False
    j = 0
    #The Q-Table learning algorithm
    while j < 500:
        j+=1
        #Choose an action using an epsilon-greedy approach
        if np.random.rand() < epsilon:
            a = 1 if np.random.uniform(0,1) >= 0.5 else 0
        else:
            a = np.argmax(Q[s,:])

        #Get new state and reward from environment
        s1_,r,terminal,_ = env.step(a)
        s1 = convert(s1_)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + gamma*np.max(Q[s1,:]) - Q[s,a])
        curr_ep_ret += r
        s = s1
        
        if terminal == True:
            #decay epsilon over time. we need to be greedy in the limit
            epsilon = epsilon*0.9995
            break
    ep_len.append(j)
    ep_ret.append(curr_ep_ret)

100%|██████████| 2000/2000 [00:05<00:00, 382.63it/s]


#### Let's test the learned policy

In [13]:
observation = env.reset()
observation = convert(observation)
episode_cnt = 0
episode_ret = 0
time_steps = 0
episode_returns_qtable = []
num_episodes = 200
while episode_cnt < num_episodes:
    # env.render()
    # Get action as given by your Q-table (function)
    action = np.argmax(Q[observation,:])
    observation, reward, done, info = env.step(action)
    observation = convert(observation)
    time_steps += 1
    episode_ret += reward
    if done:
        #Reached a terminal state
        observation = env.reset()
        observation = convert(observation)
        episode_cnt += 1
        episode_returns_qtable.append(episode_ret)
        episode_ret = 0

In [14]:
print("Q-learning training performance after 2000 episodes of training.")
print("Number of Test Episodes {}.".format(episode_cnt))
print("Expected Episode Length: {}.".format(time_steps/episode_cnt))
print("Expected Episode Return: {}.".format(np.mean(episode_returns_qtable)))

Q-learning training performance after 2000 episodes of training.
Number of Test Episodes 200.
Expected Episode Length: 67.485.
Expected Episode Return: 67.485.


# DQN

In [15]:
from collections import deque

In [22]:
MAX_EPISODE = 2000 # max number of training episodes
MAX_STEP = 200 # max number of steps in an episode

In [23]:
# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor 
EPSILON_MAX = 0.5 # starting value of epsilon
EPSILON_MIN = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch
LEARNING_RATE = 0.0001

In [24]:
replay_buffer = deque()
time_step = 0
epsilon = EPSILON_MAX
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n 

In [25]:
#Build the Q-Network
hidden_neurons = 24
state_input = tf.placeholder("float",[None,state_dim])
hidden_layer = tf.layers.dense(state_input, units=hidden_neurons, activation=tf.nn.relu)
hidden_layer = tf.layers.dense(hidden_layer, units=hidden_neurons, activation=tf.nn.relu)
Q_value = tf.layers.dense(hidden_layer, units=action_dim, activation=None)


In [26]:
action_input = tf.placeholder("float",[None,action_dim]) # one hot encoding
y_input = tf.placeholder("float",[None])
Q_action = tf.reduce_sum(tf.math.multiply(Q_value,action_input),reduction_indices = 1)
loss = tf.reduce_mean(tf.square(y_input - Q_action))
optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

In [27]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/Users/skhairy/anaconda3/envs/qopt/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [28]:
#training the agent
for episode in tqdm(range(MAX_EPISODE)):
    # Get initial State
    state = env.reset()
    # Train 
    for step in range(MAX_STEP):
        
        #Get Q(s,)
        Q_value_t = Q_value.eval(feed_dict = {state_input:[state]})[0]
        #Epsilon-Greedy action selection strategy
        if random.random() <= epsilon:
            action =  random.randint(0,action_dim - 1)
        else:
            action = np.argmax(Q_value_t)

        #Decay Epsilon over time
        epsilon -= (EPSILON_MAX - EPSILON_MIN)/MAX_EPISODE

    
        next_state,reward,done_ep,_ = env.step(action)
        # Define reward for agent
        reward_agent = -1 if done_ep else 0.1
        
        one_hot_action = np.zeros(action_dim)
        one_hot_action[action] = 1
        
        #store most recent interaction
        replay_buffer.append((state,one_hot_action,reward,next_state,done_ep))
        #if buffer is full, remove oldest interaction
        if len(replay_buffer) > REPLAY_SIZE:
            replay_buffer.popleft()

        if len(replay_buffer) > BATCH_SIZE:
            time_step += 1
            # Step 1: obtain random minibatch from replay memory
            minibatch = random.sample(replay_buffer,BATCH_SIZE)
            state_batch = [data[0] for data in minibatch]
            action_batch = [data[1] for data in minibatch]
            reward_batch = [data[2] for data in minibatch]
            next_state_batch = [data[3] for data in minibatch]
            # Step 2: calculate y
            y_batch = []
            Q_value_batch = Q_value.eval(feed_dict={state_input:next_state_batch})
            for i in range(0,BATCH_SIZE):
                done = minibatch[i][4]
                if done:
                    y_batch.append(reward_batch[i])
                else :
                    y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))

            optimizer.run(feed_dict={
                                    y_input:y_batch,
                                    action_input:action_batch,
                                    state_input:state_batch
                                    })
        
        
        state = next_state
        if done_ep:
            break

100%|██████████| 2000/2000 [03:38<00:00,  4.81it/s]


In [29]:
#TESTING LEARNED POLICY
observation = env.reset()
episode_cnt = 0
episode_ret = 0
time_steps = 0
episode_returns_DQN = []
num_episodes = 200
while episode_cnt < num_episodes:
    # env.render()
    # Get action as given by our learned policy
    action = np.argmax(Q_value.eval(feed_dict = {state_input:[observation]})[0])
    observation, reward, done, info = env.step(action)
    time_steps += 1
    episode_ret += reward
    if done:
        #Reached a terminal state
        observation = env.reset()
        episode_cnt += 1
        episode_returns_DQN.append(episode_ret)
        episode_ret = 0

In [30]:
print("DQN performance after 2000 episodes of training.")
print("Number of Test Episodes {}.".format(episode_cnt))
print("Expected Episode Length :{}.".format(time_steps/episode_cnt))
print("Expected Episode Return :{}.".format(np.mean(episode_returns_DQN)))

DQN performance after 2000 episodes of training.
Number of Test Episodes 200.
Expected Episode Length :179.51.
Expected Episode Return :179.51.


# Simple Policy Gradient

In [31]:
# hyperparameters
# number of hidden layer neurons
hidden_neurons = 10 
# every how many episodes we do parameter update
batch_size = 5 
learning_rate = 1e-2 
gamma = 0.95 
#State dimensionality
SD = env.observation_space.shape[0]

In [32]:
#Construct the policy network, which maps from states to actions.
observations_ph = tf.placeholder(tf.float32, [None,SD] , name="states")
hidden_layer = tf.layers.dense(observations_ph, units=hidden_neurons, activation=tf.nn.relu)
probability = tf.layers.dense(hidden_layer,units=1,activation=tf.nn.sigmoid)

#From here we define the parts of the network needed for learning a good policy.
action_neurons = int(np.math.log(env.action_space.n,2))
action_ph = tf.placeholder(tf.float32,[None,action_neurons], name="actions")
advantages_ph = tf.placeholder(tf.float32,name="reward_signal")

# The loss function. loglikelihood of policy * advantage
# actions with good advantage are more likely, and actions without good advantage are less likely.
loglik = tf.log(action_ph*(action_ph - probability) + (1 - action_ph)*(action_ph + probability))
loss = -tf.reduce_mean(loglik * advantages_ph) 

train_pi = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [33]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [34]:
obs_buf,reward_buf,act_buf = [],[],[]
running_reward = None
episode_ret = 0
episode_number = 1
total_episodes = 2000
episode_returns = []

init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    observation = env.reset() 
    
    pbar = tqdm(total = total_episodes)
    while episode_number <= total_episodes: 
        x = np.reshape(observation,[1,SD])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations_ph: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        obs_buf.append(x) 
        y = 1 if action == 0 else 0 # a "fake label"
        act_buf.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        episode_ret += reward

        reward_buf.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: 
            episode_number += 1
            pbar.update(1)
            # stack inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(obs_buf)
            epy = np.vstack(act_buf)
            epr = np.vstack(reward_buf)
          
            # reset array memory
            obs_buf,reward_buf,act_buf = [],[],[]
            
            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            # Normalize discounted rewards to be 0 mean, unity variance. This lowers the gradient estimator variance.
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr //= np.std(discounted_epr)
            episode_returns.append(episode_ret)

            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                #launch one training step
                sess.run(train_pi,feed_dict={observations_ph: epx, action_ph: epy, advantages_ph: discounted_epr})
                
                # Give a summary of how well our network is doing for each batch of episodes.
            if running_reward is None:
                running_reward = episode_ret
            else:
                running_reward = running_reward * 0.99 + episode_ret * 0.01
                if running_reward > 199: 
                    print("Task solved in",episode_number,'episodes!')
                    break
            
            #if (episode_number%50):
            #    print("Completed {} Episodes. Running Reward {}".format(episode_number,running_reward))         
            
            episode_ret = 0
            observation = env.reset()
            
    pbar.close()
    #TESTING LEARNED POLICY
    observation = env.reset()
    episode_cnt = 0
    episode_ret = 0
    time_steps = 0
    episode_returns_policygrad = []
    num_episodes = 200
    while episode_cnt < num_episodes:
        # env.render()
        # Get action as given by our learned policy
        x = np.reshape(observation,[1,SD])
        tfprob = sess.run(probability,feed_dict={observations_ph: x})
        action = 1 if 0.5 < tfprob else 0
        observation, reward, done, info = env.step(action)
        time_steps += 1
        episode_ret += reward
        if done:
            #Reached a terminal state
            observation = env.reset()
            episode_cnt += 1
            episode_returns_policygrad.append(episode_ret)
            episode_ret = 0

100%|██████████| 2000/2000 [00:15<00:00, 125.75it/s]


In [35]:
print("Simple Policy Gradient performance after 2000 episodes of training.")
print("Number of Test Episodes {}.".format(episode_cnt))
print("Expected Episode Length :{}.".format(time_steps/episode_cnt))
print("Expected Episode Return :{}.".format(np.mean(episode_returns_policygrad)))

Simple Policy Gradient performance after 2000 episodes of training.
Number of Test Episodes 200.
Expected Episode Length :447.945.
Expected Episode Return :447.945.
